In [3]:
import os
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [4]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY environment variable not set")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7, api_key=api_key)

In [5]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [6]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [7]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [8]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [9]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!",
 'explanation': 'This is a pun, relying on the similar sound of "fun guy" and "fungi."\n\n* **"Fun guy"** is a slang term for a fun person.\n* **"Fungi"** is the plural of "fungus," a type of organism that includes mushrooms.\n\nThe joke plays on the double meaning of "fungi."  The pizza slice is expressing liking for the mushroom, but the word "fungi" makes it sound like the pizza slice is saying the mushroom is a "fun guy," implying the mushroom is a pleasant or enjoyable companion.'}

In [10]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explanation': 'This is a pun, relying on the similar sound of "fun guy" and "fungi."\n\n* **"Fun guy"** is a slang term for a fun person.\n* **"Fungi"** is the plural of "fungus," a type of organism that includes mushrooms.\n\nThe joke plays on the double meaning of "fungi."  The pizza slice is expressing liking for the mushroom, but the word "fungi" makes it sound like the pizza slice is saying the mushroom is a "fun guy," implying the mushroom is a pleasant or enjoyable companion.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb0-3af2-605f-8002-1aa41f0d10c9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-07T09:14:10.491572+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb0-2e45-61b7-8001-c80c607a8686'}}, tasks=(), interr

In [11]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explanation': 'This is a pun, relying on the similar sound of "fun guy" and "fungi."\n\n* **"Fun guy"** is a slang term for a fun person.\n* **"Fungi"** is the plural of "fungus," a type of organism that includes mushrooms.\n\nThe joke plays on the double meaning of "fungi."  The pizza slice is expressing liking for the mushroom, but the word "fungi" makes it sound like the pizza slice is saying the mushroom is a "fun guy," implying the mushroom is a pleasant or enjoyable companion.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb0-3af2-605f-8002-1aa41f0d10c9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-07T09:14:10.491572+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb0-2e45-61b7-8001-c80c607a8686'}}, tasks=(), inter

In [12]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': "Why did the Italian chef quit his job?  Because he didn't get enough *pasta*bilities!",
 'explanation': 'The joke relies on a pun, using the similar-sounding words "pasta" and "possibilities."  The Italian chef is implying he left his job because he felt there weren\'t enough opportunities for creativity or advancement ("possibilities").  By substituting "pasta" – a key ingredient in Italian cuisine –  the joke creates a humorous and unexpected twist.'}

In [13]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explanation': 'This is a pun, relying on the similar sound of "fun guy" and "fungi."\n\n* **"Fun guy"** is a slang term for a fun person.\n* **"Fungi"** is the plural of "fungus," a type of organism that includes mushrooms.\n\nThe joke plays on the double meaning of "fungi."  The pizza slice is expressing liking for the mushroom, but the word "fungi" makes it sound like the pizza slice is saying the mushroom is a "fun guy," implying the mushroom is a pleasant or enjoyable companion.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb0-3af2-605f-8002-1aa41f0d10c9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-07T09:14:10.491572+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb0-2e45-61b7-8001-c80c607a8686'}}, tasks=(), interr

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why does the pizza slice like the mushroom so much?\n\nBecause he's a fungi!", 'explanation': 'This is a pun, relying on the similar sound of "fun guy" and "fungi."\n\n* **"Fun guy"** is a slang term for a fun person.\n* **"Fungi"** is the plural of "fungus," a type of organism that includes mushrooms.\n\nThe joke plays on the double meaning of "fungi."  The pizza slice is expressing liking for the mushroom, but the word "fungi" makes it sound like the pizza slice is saying the mushroom is a "fun guy," implying the mushroom is a pleasant or enjoyable companion.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb0-3af2-605f-8002-1aa41f0d10c9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-07T09:14:10.491572+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb0-2e45-61b7-8001-c80c607a8686'}}, tasks=(), inter

### Time Travel

In [15]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [16]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

EmptyInputError: Received no input for __start__

In [ ]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the mushroom go to the pizza party? Because he was a fungi and everyone wanted a pizza him!', 'explanation': 'This joke plays on the word "fun guy" (fungi) which sounds like "fungi," a type of mushroom. The play on words is that the mushroom went to the pizza party because he was a "fun guy" and people wanted to "pizza" (see) him. The joke is a pun that combines the idea of mushrooms being fungi with the concept of being a fun person at a party.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc70-100a-6bff-8002-7d6c3d37b1f4'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-29T21:57:21.959833+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc70-064c-630b-8001-707d60a085ad'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the mushroom go to the 

#### Updating State

In [ ]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f06cc72-ca16-6359-8001-7eea05e07dd2'}}

In [ ]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc72-ca16-6359-8001-7eea05e07dd2'}}, metadata={'source': 'update', 'step': 1, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-29T21:58:35.155132+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, tasks=(PregelTask(id='0f085bb0-c1e8-d9fd-fb15-c427126b7cd6', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the mushroom go to the pizza party? Because he was a fungi and everyone wanted a pizza him!', 'explanation': 'This joke plays on the word "fun guy" (fungi) which sounds like "fungi," a type of mushroom. The play on words is that the mushroom went to the pizza party because he was a "fun guy" and people 

In [ ]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc72-ca16-6359-8001-7eea05e07dd2"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa bring a ladder to the party? \nBecause it wanted to be the best snack in the room and rise to the occasion!',
 'explanation': 'This joke plays on the double meaning of the word "rise." In one sense, "rise" means to physically move upwards, which is why the samosa brought a ladder to the party. However, in another sense, "rise" can also mean to perform well or excel, as in rising to the occasion. So, the samosa brought a ladder to symbolize its desire to physically rise above the other snacks at the party and also to metaphorically rise to the occasion by being the best snack in the room.'}

In [ ]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa bring a ladder to the party? \nBecause it wanted to be the best snack in the room and rise to the occasion!', 'explanation': 'This joke plays on the double meaning of the word "rise." In one sense, "rise" means to physically move upwards, which is why the samosa brought a ladder to the party. However, in another sense, "rise" can also mean to perform well or excel, as in rising to the occasion. So, the samosa brought a ladder to symbolize its desire to physically rise above the other snacks at the party and also to metaphorically rise to the occasion by being the best snack in the room.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc75-4407-6195-8003-b08dcfd27511'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-29T21:59:41.628661+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoin

### Fault Tolerance

In [17]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [18]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [19]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [20]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [21]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [23]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb1-9889-69e6-8001-bed25cedcb75'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-09-07T09:14:47.148983+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb1-9886-6077-8000-80377efb6c93'}}, tasks=(PregelTask(id='1f3be647-b26e-1e9f-d047-311695e5e9be', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'start'}, next=('step_1',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f08bcb1-9886-6077-8000-80377efb6c93'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-07T09:14:47.147517+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'ch